Import necessary libraries

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# DATA PREPARATION

Load Data

In [24]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [25]:
X_train, X_test = X_train/255, X_test/255

Orders of the class names in the dataset

In [26]:
class_order = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [27]:
# Create a list to store the ordered training items
ordered_train_images = []
ordered_train_labels = []

# Iterate over the class order and extract the corresponding training items
for i in range(len(class_order)):
    class_name = class_order[i]
    class_index = i
    class_indices = np.where(y_train == class_index)[0]
    class_images = X_train[class_indices]
    class_labels = y_train[class_indices]
    ordered_train_images.extend(class_images)
    ordered_train_labels.extend(class_labels)
# Convert the ordered training items back to numpy arrays
X_train = np.array(ordered_train_images)
y_train = np.array(ordered_train_labels)

In [28]:
len(X_train)

50000

Select samples from train dataset

In [29]:
ranges = [(0, 1000), (5000, 6000), (10000, 11000), (15000, 16000),
          (20000, 21000), (25000, 26000), (30000, 31000),
          (35000, 36000), (40000, 41000), (45000, 46000)]

selected_indices = []
for start, end in ranges:
    selected_indices.extend(range(start, start + 2))

X_train = X_train[selected_indices]
y_train = y_train[selected_indices]

Select test samples

In [30]:
# X_test = X_test[:383]
# y_test = y_test[:383]

# step2

Load model for style transfer

In [31]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

Load images

In [32]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

style_images = [load_image("../styles/a4.png"), load_image("../styles/frida.jpg"), load_image("../styles/p1.jpg")]

Add content images to an array

In [33]:
content_images = []

for i in range(20):
    content = tf.image.convert_image_dtype(X_train[i], tf.float32)
    content = content[tf.newaxis, :]
    content_images.append(content)

In [34]:
def generate_samples(content_images, style_images, style_transfer_model):
    negative_samples = []
    positive_samples = []
    # Generate positive samples
    for content in content_images:

        for style in style_images:
            stylized_image = style_transfer_model(tf.constant(content), tf.constant(style))[0]
            positive_samples.append((content, stylized_image))
    
    # Generate negative samples
    for content1 in content_images:
        for content2 in content_images:
            if  not (np.array_equal(content1, content2)):
                random_style = style_images[np.random.randint(0, len(style_images)-1)]
                stylized_image = style_transfer_model(tf.constant(content1), tf.constant(random_style))[0]
                negative_samples.append((content1, stylized_image))
    
    return positive_samples, negative_samples

In [35]:
# Define the contrastive model architecture
def create_contrastive_model(input_shape_content, input_shape_style, embedding_dim):
    input_content = tf.keras.Input(shape=input_shape_content)
    input_style = tf.keras.Input(shape=input_shape_style)
    
    # Content branch
    x_content = layers.Conv2D(32, (3, 3), activation='relu')(input_content)
    x_content = layers.MaxPooling2D((2, 2))(x_content)
    x_content = layers.Conv2D(64, (3, 3), activation='relu')(x_content)
    x_content = layers.MaxPooling2D((2, 2))(x_content)
    x_content = layers.Flatten()(x_content)
    x_content = layers.Dense(embedding_dim, activation='relu')(x_content)
    
    # Style branch
    x_style = layers.Conv2D(32, (3, 3), activation='relu')(input_style)
    x_style = layers.MaxPooling2D((2, 2))(x_style)
    x_style = layers.Conv2D(64, (3, 3), activation='relu')(x_style)
    x_style = layers.MaxPooling2D((2, 2))(x_style)
    x_style = layers.Flatten()(x_style)
    x_style = layers.Dense(embedding_dim, activation='relu')(x_style)
    
    # Concatenate content and style embeddings
    concatenated = layers.Concatenate()([x_content, x_style])
    
    flattened = layers.Flatten()(concatenated)
    # Normalize the concatenated embeddings
    normalized = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(concatenated)
    
    # Create the model with inputs and outputs
    model = tf.keras.Model(inputs=[input_content, input_style], outputs=normalized)
    return model


def contrastive_loss(y_true, y_pred, margin=1.0):
    square_pred = tf.square(1 - y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

In [36]:
positive_samples, negative_samples = generate_samples(content_images, style_images, model)

In [37]:
contrastive_model = create_contrastive_model(input_shape_content=(32, 32, 3),
                                             input_shape_style=(32, 32, 3),
                                             embedding_dim=128)

In [38]:
positive_samples = np.array(positive_samples)
negative_samples = np.array(negative_samples)

In [39]:
print(f"p:{len(positive_samples)} | n: {len(negative_samples)}")

p:60 | n: 380


In [40]:
content_images_positive = positive_samples[:, 0]
stylized_images_positive = positive_samples[:, 1]
content_images_negative = negative_samples[:, 0]
stylized_images_negative = negative_samples[:, 1]

In [41]:
positive_labels = np.ones(len(positive_samples))
negative_labels = np.zeros(len(negative_samples))

In [42]:
train_images_content = np.concatenate((content_images_positive, content_images_negative))
train_images_style = np.concatenate((stylized_images_positive, stylized_images_negative))
train_labels_content = np.concatenate((positive_labels, negative_labels))

train_images_content = np.squeeze(train_images_content, axis=1)
train_images_style = np.squeeze(train_images_style, axis=1)

#test data
# indices = np.random.permutation(len(20))
# test_images_style = train_images_style[indices]

# Shuffle the data
indices = np.random.permutation(len(train_images_content))
train_images_content = train_images_content[indices]
train_images_style = train_images_style[indices]
train_labels_content = train_labels_content[indices]



# Prepare the training data
train_data = ([train_images_content, train_images_style], train_labels_content)

# Split the train_data into content and style inputs
train_content_images = train_data[0][0]
train_style_images = train_data[0][1]
train_labels = train_data[1]


In [ ]:
train_images_content.shape

In [ ]:
train_style_images.shape

In [ ]:
X_test.shape

In [ ]:
train_labels.shape

In [ ]:
y_test.shape

In [ ]:
contrastive_model.compile(optimizer='adam', loss=contrastive_loss)
contrastive_model.fit([train_content_images, train_style_images], train_labels, epochs=10)

In [ ]:
len(X_train)

In [ ]:
train_images_style = train_images_style[:]

In [ ]:
# Freeze the encoder model
contrastive_model.trainable = False

# Add a new classifier layer
output_layer = tf.keras.layers.Dense(10, activation='softmax')(contrastive_model.output)

# Define the new model for classification
classification_model = Model(contrastive_model.input, output_layer)

classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the classification model
classification_model.fit(train_data[0], train_data[1], epochs=10,\
                         validation_data=([X_test,train_images_style],y_test),\
                         validation_batch_size =20)

In [ ]:
train_images_style.shape

In [ ]:
y_test

In [ ]:
train_images_style.shape